In [20]:
from dotenv import load_dotenv
load_dotenv()

True

In [21]:
import json
import pandas as pd
from pathlib import Path
from copy import deepcopy
from functools import partial

from bellek.utils import set_seed, jprint
from bellek.musique.singlehop import benchmark
from bellek.musique.qa import answer_question_cte

# set_seed(89)

In [22]:
from tqdm.auto import tqdm
tqdm.pandas()

In [23]:
df = pd.read_json('../../data/generated/musique-common/base-dataset-train.jsonl', orient='records', lines=True)
# df = df.sample(200, random_state=31)

print(df.shape)
df.head()

(14376, 8)


,id,paragraphs,question,question_decomposition,answer,answer_aliases,answerable,answers
0,2hop__128801_205185,"[{'idx': 0, 'title': 'Pama, Burkina Faso', 'pa...",What county is the town where KNFM is licensed...,"[{'id': 128801, 'question': 'What town is KNFM...",Midland County,"[Midland County, Midland County, Texas]",True,"[Midland County, Midland County, Texas]"
1,2hop__719559_217649,"[{'idx': 0, 'title': 'Antoine Marchand', 'para...",What's the record label of the artist who put ...,"[{'id': 719559, 'question': 'Me and Julio Down...",Warner Bros.,[Warner Bros.],True,[Warner Bros.]
2,2hop__128806_205185,"[{'idx': 0, 'title': 'Spanish Town', 'paragrap...",What region is the town where KQRX is liscense...,"[{'id': 128806, 'question': 'What town is KQRX...",Midland County,"[Midland County, Midland County, Texas]",True,"[Midland County, Midland County, Texas]"
3,2hop__837090_278127,"[{'idx': 0, 'title': 'The Opening (album)', 'p...",What is the record label of the Do It Again pe...,"[{'id': 837090, 'question': 'Do It Again >> pe...",Roc-A-Fella Records,[Roc-A-Fella Records],True,[Roc-A-Fella Records]
4,2hop__128895_11424,"[{'idx': 0, 'title': 'Ehrhardt, South Carolina...",How many households were there in the town WPU...,"[{'id': 128895, 'question': 'What town is WPUR...","15,504","[15,504]",True,"[15,504]"


In [26]:
df.set_index('id', drop=False).loc[['2hop__834974_332063']]

,id,paragraphs,question,question_decomposition,answer,answer_aliases,answerable,answers
id,,,,,,,,
2hop__834974_332063,2hop__834974_332063,"[{'idx': 0, 'title': 'Kingdom of Gera', 'parag...",Which region shares border with one where Perr...,"[{'id': 834974, 'question': 'Perry Township >>...",Franklin County,[Franklin County],True,[Franklin County]


In [5]:
perfect_retrieval_func = lambda docs, query: [doc for doc in docs if doc['is_supporting']]

In [6]:
qa_func = partial(answer_question_cte, model_name="gpt-3.5-turbo")
df_gpt_35, scores = benchmark(df, qa_func, perfect_retrieval_func, ignore_errors=False)

  0%|          | 0/200 [00:00<?, ?it/s]

In [7]:
qa_func = partial(answer_question_cte, model_name="gpt-4-turbo")
df_gpt_4, scores = benchmark(df, qa_func, perfect_retrieval_func, ignore_errors=False)

  0%|          | 0/200 [00:00<?, ?it/s]

In [8]:
df_gpt_35['exact_match'].sum(), df_gpt_4['exact_match'].sum()

(128, 124)

In [9]:
df_gpt_35['fuzzy_match'].sum(), df_gpt_4['fuzzy_match'].sum()

(161, 170)

In [10]:
df_gpt_4.iloc[0]['raw_output']

{'answer': 'Gainesville, Florida',
 'hops': [{'question': 'What is the birthplace of the artist who played Lucky on King of the Hill?',
   'query': 'What is the birthplace of the artist who played Lucky on King of the Hill?',
   'context': '# Tom Petty and the Heartbreakers\nTom Petty and the Heartbreakers (alternately Tom Petty & The Heartbreakers) were an American rock band from Gainesville, Florida. Formed in 1976, the band originally comprised Tom Petty (lead singer, guitar), Mike Campbell (lead guitarist), Ron Blair (bass guitar), Stan Lynch (drums), and Benmont Tench (keyboards). In 1981, Blair, weary of the touring lifestyle, departed the band. His replacement, Howie Epstein, stayed with the band for the next two decades. In 1991, Scott Thurston joined the band as a multi-instrumentalist—mostly on rhythm guitar and second keyboards. Blair returned to the Heartbreakers in 2002, the year before Epstein\'s death. In 1994, Steve Ferrone replaced Lynch on drums. The band is best know

In [11]:
mask = (~df_gpt_35['fuzzy_match']) & df_gpt_4['fuzzy_match']
mask.sum()

21

In [12]:
examples = []
for idx, row in df_gpt_4.loc[mask].iterrows():
    id = row['id']
    raw_output = row['raw_output']
    examples.append({
        "id": id,
        "context": raw_output['hops'][0]['context'],
        "question": raw_output['hops'][0]['question'],
        "generation": raw_output['hops'][0]['llm_output']['generation'],
    })
examples

[{'id': '2hop__360010_159937',
  'context': '# Sadad, Bahrain\nSadad is a coastal village situated on the western shore of Bahrain. It is situated to the south of Malkiya and to the west of Hamad Town, in the Northern Governorate administrative region of the country.\n# Bahrain World Trade Center\nThe Bahrain World Trade Center (also called Bahrain WTC or BWTC) is a 240-metre-high (787 ft), 50-floor, twin tower complex located in Manama, Bahrain. Designed by the multi-national architectural firm Atkins, construction on the towers was completed in 2008. It is the first skyscraper in the world to integrate wind turbines into its design. The wind turbines were developed, built and installed by the Danish company Norwin A/S.The structure is constructed close to the King Faisal Highway, near popular landmarks such as the towers of Bahrain Financial Harbour (BFH), NBB and Abraj Al Lulu. It currently ranks as the second-tallest building in Bahrain, after the twin towers of the Bahrain Financi

In [13]:
with open('cte-few-shot-examples.json', 'w') as f:
    json.dump(examples, f, indent=2)